In [2]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

os.makedirs("data/light", exist_ok=True)
os.makedirs("data/dark", exist_ok=True)

def ensure_sample_images():
    if len(glob.glob("data/light/*.jpg")) == 0:
        white_face = np.ones((64, 64, 3), dtype=np.uint8) * 255
        cv2.imwrite("data/light/fake1.jpg", white_face)
    if len(glob.glob("data/dark/*.jpg")) == 0:
        black_face = np.zeros((64, 64, 3), dtype=np.uint8)
        cv2.imwrite("data/dark/fake1.jpg", black_face)

def load_images_from_folder(folder):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.jpg')):
        img = cv2.imread(filename)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            images.append(img.flatten())
    return images

def prepare_dataset():
    ensure_sample_images()
    light_faces = load_images_from_folder("data/light")
    dark_faces = load_images_from_folder("data/dark")
    if len(light_faces) == 0 or len(dark_faces) == 0:
        raise ValueError("Image folders must contain valid .jpg images.")
    X = np.array(light_faces + dark_faces)
    y = np.array(['light'] * len(light_faces) + ['dark'] * len(dark_faces))
    return X, y

def train_knn_model(X, y, k=3):
    k = min(k, len(y))
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X, y)
    preds = model.predict(X)
    print("Training Results:\n", classification_report(y, preds))
    return model

def capture_image(save_path="data/captured_test.jpg"):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Could not open webcam.")
        return False
    print("Press SPACE to capture, ESC to cancel.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow("Webcam - Press SPACE to Capture", frame)
        key = cv2.waitKey(1)
        if key % 256 == 27:
            print("Cancelled.")
            break
        elif key % 256 == 32:
            resized = cv2.resize(frame, (64, 64))
            cv2.imwrite(save_path, resized)
            print(f"Image saved to {save_path}")
            break
    cap.release()
    cv2.destroyAllWindows()
    return os.path.exists(save_path)

def predict_face(model, image_path):
    if not os.path.exists(image_path):
        print("Image not found.")
        return
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (64, 64))
    flat = img_resized.flatten().reshape(1, -1)
    prediction = model.predict(flat)[0]
    print(f"Prediction: {prediction}")
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f"Predicted: {prediction}")
    plt.axis('off')
    plt.show()

if __name__ == "__main__":
    try:
        X, y = prepare_dataset()
        model = train_knn_model(X, y)
        if capture_image():
            predict_face(model, "data/captured_test.jpg")
    except Exception as e:
        print(f"Error: {e}")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Training Results:
               precision    recall  f1-score   support

        dark       0.50      1.00      0.67         1
       light       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Press SPACE to capture, ESC to cancel.


KeyboardInterrupt: 